In [ ]:
!pip install natsort

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import joblib
import sys
from natsort import natsorted
sys.modules['sklearn.externals.joblib'] = joblib
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import itertools
import seaborn as sns
import zipfile
import os
import cv2
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.svm import SVC
from mlxtend.feature_selection import SequentialFeatureSelector as SFS

In [ ]:
from tensorflow.keras.utils import to_categorical
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D,MaxPooling2D,Activation, Dropout, Flatten, Dense
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
!mkdir defect_bearing_saved_pictures

In [ ]:
!cp /content/drive/MyDrive/defect_bearing_saved_pictures_one_by_one/Pictures_generated_one_by_one_bearing_defaults_morefeatures_norma.zip /content/sample_data

In [ ]:

with zipfile.ZipFile("/content/sample_data/Pictures_generated_one_by_one_bearing_defaults_morefeatures_norma.zip","r") as zip_ref:
    zip_ref.extractall("/content/defect_bearing_saved_pictures")


In [ ]:
def union_lists(*lists):
  # use the chain() function from itertools to concatenate all the lists
  concatenated_list = list(itertools.chain(*lists))
  # use the set() function to remove duplicates and convert the concatenated list to a set
  unique_set = set(concatenated_list)
  # convert the set back to a list and return it as the final union
  final_union = list(unique_set)
  final_union.sort()
  return final_union

In [ ]:

class reading_data_picture():
    def __init__(self, path, img_size_x, img_size_y):
        self.path = path
        self.img_size_x = img_size_x
        self.img_size_y = img_size_y
    def get_data(self):
        data = []
        picture_list = os.listdir(self.path)
        picture_list = natsorted(picture_list)
        for x in picture_list:
            picture_path = os.path.join(self.path,x)
            out = cv2.imread(picture_path)
            out_resize = cv2.resize(out,(self.img_size_x, self.img_size_y))
            data.append(out_resize)
        data_out = np.array(data) /255
        return data_out

In [ ]:
data_total  = reading_data_picture('/content/defect_bearing_saved_pictures', 224, 224)
data_picture = data_total.get_data()

In [ ]:
class reading_data():
  def __init__(self, path):
    self.path = path
  def reading_data(self):
    df = pd.read_csv(self.path)
    df['displacemednt'] = df['Acc, Rms (RMS)'] / df['Vel, Rms (RMS)']
    df['momentum'] =  df['Acc, Rms (RMS)'] * df['Vel, Rms (RMS)']
    print(df.isnull().sum())
    if df.isnull().sum().all():
      df = df.fillna(df.iloc[0])
    else:
      pass
    outlier_indices_Vel = np.where((df['Vel, Rms (RMS)'] > 6.07) & (df['Vel, Peak (RMS)'] > 17.33) & (df['Vel, Peak to peak (RMS)'] > 32.70) )
    outlier_indices_ace = np.where((df['Acc, Rms (RMS)'] > 2.32) )
    outlier_indices_crest = np.where((df['Crest (RMS)'] > 4.74) )
    outlier_indices_Kurt = np.where((df['Kurt (RMS)'] > 0.81) )
    outliars =union_lists(list(outlier_indices_Vel[0]), list(outlier_indices_ace[0]), list(outlier_indices_crest[0]), list(outlier_indices_Kurt[0]))
    no_outliers = df.drop(outliars)
    return no_outliers
  def visualization_pearson(self):
    df = pd.read_csv(self.path)
    corr =df.corr()
    return sns.heatmap(
        corr, annot=True,
        vmin=-1, vmax=1, center=0,
        cmap=sns.diverging_palette(5, 220, n=50),
        square=True)
  def visualization_kendas(self):
    df = pd.read_csv(self.path)
    corr =df.corr(method='kendall')
    return sns.heatmap(
        corr, annot=True,
        center=0,
        cmap=sns.diverging_palette(5, 220, n=50),
        square=True)
  def visualization_spearman(self):
    df = pd.read_csv(self.path)
    corr =df.corr(method='spearman')
    return sns.heatmap(
        corr, annot=True,
        center=0,
        cmap=sns.diverging_palette(5, 220, n=50),
        square=True)



In [ ]:
from imblearn.combine import SMOTETomek
from imblearn.combine import SMOTEENN
from imblearn.over_sampling import BorderlineSMOTE

In [ ]:
class augmnentation():
  def __init__(self,x,y):
    self.x = x
    self.y = y
  def SMOTETomek(self):
    print(len(self.x))
    if len(self.x.shape) == 4:
      x_reshaped = self.x.reshape( self.x.shape[0], self.x.shape[1]* self.x.shape[2] *  self.x.shape[-1] )
      print(x_reshaped.shape)
      smt = SMOTETomek(sampling_strategy='auto')
      X_resampled, y_resampled = smt.fit_resample(x_reshaped, self.y)
      X_resampled_re = X_resampled.reshape(X_resampled.shape[0],self.x.shape[1], self.x.shape[2],self.x.shape[-1])
      print("The distribution over the final distribution ", pd.DataFrame(y_resampled).value_counts())
      return X_resampled_re, y_resampled
    if len(self.x.shape) == 2:
      smt = SMOTETomek(sampling_strategy='auto')
      X_resampled, y_resampled = smt.fit_resample(self.x, self.y)
      return  X_resampled, y_resampled
  def SMOTEENN(self):
    if len(self.x.shape) == 4:
      x_reshaped = self.x.reshape( self.x.shape[0], self.x.shape[1]* self.x.shape[2] *  self.x.shape[-1]  )
      smote_enn = SMOTEENN()
      X_resampled, y_resampled = smote_enn.fit_resample(x_reshaped, self.y)
      X_resampled_re = X_resampled.reshape(X_resampled.shape[0],self.x.shape[1], self.x.shape[2],self.x.shape[-1])
      print("The distribution over the final distribution ", pd.DataFrame(y_resampled).value_counts())
      return X_resampled_re, y_resampled
    if len(self.x.shape) == 2:
      smt = SMOTETomek(sampling_strategy='auto')
      X_resampled, y_resampled = smote_enn.fit_resample(self.x, self.y)
      return  X_resampled, y_resampled
  def BorderlineSMOTE(self):
    if len(self.x.shape) == 4:
      x_reshaped = self.x.reshape( self.x.shape[0], self.x.shape[1]* self.x.shape[2] *  self.x.shape[-1]  )
      blsmote = BorderlineSMOTE(sampling_strategy='minority', kind='borderline-1')
      X_resampled, y_resampled = blsmote.fit_resample(x_reshaped, self.y)
      X_resampled_re = X_resampled.reshape(X_resampled.shape[0],self.x.shape[1], self.x.shape[2],self.x.shape[-1])
      print("The distribution over the final distribution ", pd.DataFrame(y_resampled).value_counts())
      return X_resampled_re, y_resampled
    if len(self.x.shape) == 2:
      blsmote = BorderlineSMOTE(sampling_strategy='minority', kind='borderline-1')
      X_resampled, y_resampled = blsmote.fit_resample(self.x, self.y)
      return  X_resampled, y_resampled

In [ ]:

data = reading_data("/content/bearing-failure.csv")
df = data.reading_data()
target = df['Label']

COMP_NAME                  0
Vel, Rms (RMS)             0
Acc, Rms (RMS)             0
Crest (RMS)                0
Kurt (RMS)                 0
Vel, Peak (RMS)            0
Vel, Peak to peak (RMS)    0
MP_LOC                     0
Label                      0
displacemednt              0
momentum                   0
dtype: int64


In [ ]:
target.shape

(1242,)

In [ ]:
data_picture.shape

(1242, 224, 224, 3)

In [ ]:
aug = augmnentation(data_picture,np.array(target))
x_re , y_re = aug.SMOTETomek()


1242
(1242, 150528)
The distribution over the final distribution  0
2    342
0    294
1    291
Name: count, dtype: int64


In [ ]:
x_re1 , y_re1 = aug.SMOTEENN()

The distribution over the final distribution  0
2    83
0    53
1    40
Name: count, dtype: int64


In [ ]:
#x_re2 , y_re2 = aug.BorderlineSMOTE()

In [ ]:
data_augx , data_augy = pd.concat([pd.DataFrame(x_re.reshape(x_re.shape[0],x_re.shape[1]*x_re.shape[2]*x_re.shape[-1] )),pd.DataFrame(x_re1.reshape(x_re1.shape[0],x_re1.shape[1]*x_re1.shape[2]*x_re1.shape[-1]))], ignore_index=True),pd.concat([pd.DataFrame(y_re), pd.DataFrame(y_re1)], ignore_index=True)
data_augx = np.array(data_augx).reshape(data_augx.shape[0],x_re.shape[1], x_re.shape[2],x_re.shape[-1])
data_augy = np.array(data_augy)

In [ ]:
from sklearn.preprocessing import OneHotEncoder
encoder = OneHotEncoder(sparse_output=False)
X_train, X_test, y_train, y_test = train_test_split(data_augx, encoder.fit_transform(data_augy.reshape( -1,1)), test_size=0.20, random_state=0)
print("The training shaps is : ", X_train.shape)
print("The testing shaps is : ", X_test.shape)
print("The training target is : ", y_train.shape)
print("The testing target is : ", y_test.shape)

The training shaps is :  (882, 224, 224, 3)
The testing shaps is :  (221, 224, 224, 3)
The training target is :  (882, 3)
The testing target is :  (221, 3)


In [ ]:
cnn_model = Sequential()


pretrained_model= tf.keras.applications.Xception(
    include_top=False,
    weights= 'imagenet',
    input_shape= (224,224,3),
    pooling= 'avg',
    classes= 3,
)


#Freezing the deeper layers
for layer in pretrained_model.layers:
        layer.trainable=False

#adding our layers to the model
cnn_model.add(pretrained_model)
cnn_model.add(Flatten())
cnn_model.add(Dropout(0.2))
cnn_model.add(Dense(256, activation='swish'))
cnn_model.add(Dropout(0.2))
cnn_model.add(Dense(64, activation='swish'))
cnn_model.add(Dropout(0.2))
cnn_model.add(Dense(3, activation='softmax'))


83683744/83683744 ━━━━━━━━━━━━━━━━━━━━ 5s 0us/step


In [ ]:
early_stopping_cb =  tf.keras.callbacks.EarlyStopping(patience=1000,restore_best_weights=True)
lr_scheduler1 = tf.keras.callbacks.ReduceLROnPlateau(factor=0.333, patience=300)
checkpoint_cb = tf.keras.callbacks.ModelCheckpoint("Bearing_defect_detetcion.keras",save_best_only=True)

In [ ]:
METRICS = [
      keras.metrics.TruePositives(name='tp'),
      keras.metrics.TrueNegatives(name='tn'),
      keras.metrics.FalsePositives(name='fp'),
      keras.metrics.FalseNegatives(name='fn'),
      keras.metrics.BinaryAccuracy(name='accuracy'),
      keras.metrics.Precision(name='precision'),
      keras.metrics.Recall(name='recall'),
      keras.metrics.AUC(name='auc'),
]

In [ ]:

cnn_model.compile(optimizer='adamw', loss=tf.keras.losses.CategoricalFocalCrossentropy(),metrics=METRICS)

In [ ]:
history = cnn_model.fit(X_train,y_train, epochs=2000, batch_size = 32,
                        validation_data=(X_test, y_test), validation_batch_size = 32,
                  callbacks=[early_stopping_cb,lr_scheduler1,checkpoint_cb])

28/28 ━━━━━━━━━━━━━━━━━━━━ 5s 160ms/step - accuracy: 0.9079 - auc: 0.9672 - fn: 78.3793 - fp: 49.4828 - loss: 0.0287 - precision: 0.8866 - recall: 0.8295 - tn: 906.3793 - tp: 399.5517 - val_accuracy: 0.8205 - val_auc: 0.8492 - val_fn: 71.0000 - val_fp: 48.0000 - val_loss: 0.1628 - val_precision: 0.7576 - val_recall: 0.6787 - val_tn: 394.0000 - val_tp: 150.0000 - learning_rate: 1.1089e-04
Epoch 690/2000
28/28 ━━━━━━━━━━━━━━━━━━━━ 5s 160ms/step - accuracy: 0.9196 - auc: 0.9739 - fn: 73.7931 - fp: 43.7931 - loss: 0.0243 - precision: 0.9051 - recall: 0.8476 - tn: 912.0690 - tp: 404.1379 - val_accuracy: 0.8084 - val_auc: 0.8519 - val_fn: 71.0000 - val_fp: 56.0000 - val_loss: 0.1610 - val_precision: 0.7282 - val_recall: 0.6787 - val_tn: 386.0000 - val_tp: 150.0000 - learning_rate: 1.1089e-04
Epoch 691/2000
28/28 ━━━━━━━━━━━━━━━━━━━━ 6s 180ms/step - accuracy: 0.9314 - auc: 0.9782 - fn: 66.7586 - fp: 33.4828 - loss: 0.0223 - precision: 0.9284 - recall: 0.8606 - tn: 922.3793 - tp: 411.1724 - va